References :   
https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24  
https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb

In [122]:
import pandas as pd
data_1 = pd.read_excel('./TBBT_Tivo.xlsx', error_bad_lines=False);
data_1['name'] = 'TBBT'
data_2 = pd.read_excel('./FIUP_Tivo.xlsx', error_bad_lines=False);
data_2['name'] = 'FIUP'
data_3 = pd.read_excel('./Ellen_Tivo.xlsx', error_bad_lines=False);
data_3['name'] = 'ELLEN'
data = data_1.append(data_2,ignore_index = False) 
data = data.append(data_3,ignore_index = False) 

data.head()

,description,name
0,Two nerdy physicists share an apartment and an...,TBBT
1,Leonard and Sheldon meet their beautiful new n...,TBBT
2,The guys camp out in a line hoping to see an a...,TBBT
3,Howard and Raj try to discover where Sheldon d...,TBBT
4,"After the guys, except for Sheldon, go camping...",TBBT


In [123]:
data_text = data[['description','name']]
data_text['index'] = data_text.index
documents = data_text

In [124]:
print(len(documents))
print(documents[:5])


668
                                         description  name  index
0  Two nerdy physicists share an apartment and an...  TBBT      0
1  Leonard and Sheldon meet their beautiful new n...  TBBT      1
2  The guys camp out in a line hoping to see an a...  TBBT      2
3  Howard and Raj try to discover where Sheldon d...  TBBT      3
4  After the guys, except for Sheldon, go camping...  TBBT      4


In [125]:
documents[documents.index==0]

,description,name,index
0,Two nerdy physicists share an apartment and an...,TBBT,0
0,A house with three to four bedrooms and a spac...,FIUP,0
0,Ellen's flair for fun hits primetime with supe...,ELLEN,0


In [126]:
# !pip install gensim

In [127]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
from nltk import stem
import numpy as np
np.random.seed(2018)
import nltk
nltk.download('wordnet')
stemmer = SnowballStemmer("english")

[nltk_data] Downloading package wordnet to /Users/krishna/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [128]:
# print(" ".join(SnowballStemmer.languages))

In [129]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))
def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [130]:
doc_sample = documents[documents['index'] == 0].values[0][0]
print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
['Two', 'nerdy', 'physicists', 'share', 'an', 'apartment', 'and', 'an', 'unlikely', 'friendship', 'with', 'their', 'beautiful', 'neighbor', 'with', 'whom', 'one', 'of', 'them', 'is', 'infatuated.', 'Like', 'the', 'universe', 'after', 'the', 'big', 'bang,', 'the', "show's", 'popularity', 'expanded,', 'thanks', 'to', 'breakout', 'star', 'Jim', 'Parsons,', 'along', 'with', 'the', 'chemistry', 'among', 'the', 'friends', 'and', 'the', 'developing', 'romance', 'between', 'Leonard', 'and', 'Penny.', 'The', 'addition', 'of', 'Melissa', 'Rauch', 'and', 'Mayim', 'Bialik', 'in', 'later', 'seasons', 'also', 'enhanced', 'the', 'stories', 'and', 'relationships', 'of', 'the', 'leads.']


 tokenized and lemmatized document: 
['nerdi', 'physicist', 'share', 'apart', 'unlik', 'friendship', 'beauti', 'neighbor', 'infatu', 'like', 'univers', 'bang', 'popular', 'expand', 'thank', 'breakout', 'star', 'parson', 'chemistri', 'friend', 'develop', 'romanc', 'leonard', 'penni', 'addit', 'meli

In [131]:
processed_docs = documents['description'].map(preprocess)
processed_docs[:10]

0    [nerdi, physicist, share, apart, unlik, friend...
1    [leonard, sheldon, meet, beauti, neighbor, inv...
2    [guy, camp, line, hop, anticip, movi, penni, b...
3    [howard, discov, sheldon, disappear, afternoon...
4    [guy, sheldon, camp, penni, injur, leav, sheld...
5    [priya, use, legal, expertis, apart, sheldon, ...
6    [attract, penni, boyfriend, howard, argu, bett...
7    [howard, baffl, sheldon, accept, apolog, agent...
8    [second, season, open, penni, surpris, turn, s...
9    [guy, road, trip, comic, book, convent, bakers...
Name: description, dtype: object

In [132]:
dictionary = gensim.corpora.Dictionary(processed_docs)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break
len(dictionary)

0 addit
1 apart
2 bang
3 beauti
4 bialik
5 breakout
6 chemistri
7 develop
8 enhanc
9 expand
10 friend


3812

In [133]:
dictionary[12]

'infatu'

In [134]:
dictionary.filter_extremes(no_below=2, no_above=0.5, keep_n=100000)

In [135]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[1]

[(3, 1),
 (9, 2),
 (11, 1),
 (24, 1),
 (25, 1),
 (26, 1),
 (27, 1),
 (28, 1),
 (29, 1),
 (30, 1),
 (31, 1),
 (32, 1),
 (33, 1),
 (34, 1),
 (35, 1),
 (36, 1),
 (37, 1),
 (38, 1),
 (39, 1)]

In [136]:
bow_doc_4310 = bow_corpus[0]
for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 0 ("addit") appears 1 time.
Word 1 ("apart") appears 1 time.
Word 2 ("bang") appears 1 time.
Word 3 ("beauti") appears 1 time.
Word 4 ("develop") appears 1 time.
Word 5 ("friend") appears 1 time.
Word 6 ("friendship") appears 1 time.
Word 7 ("later") appears 1 time.
Word 8 ("lead") appears 1 time.
Word 9 ("leonard") appears 1 time.
Word 10 ("like") appears 1 time.
Word 11 ("neighbor") appears 1 time.
Word 12 ("parson") appears 1 time.
Word 13 ("penni") appears 1 time.
Word 14 ("physicist") appears 1 time.
Word 15 ("popular") appears 1 time.
Word 16 ("relationship") appears 1 time.
Word 17 ("romanc") appears 1 time.
Word 18 ("season") appears 1 time.
Word 19 ("share") appears 1 time.
Word 20 ("star") appears 1 time.
Word 21 ("stori") appears 1 time.
Word 22 ("univers") appears 1 time.
Word 23 ("unlik") appears 1 time.


In [137]:
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.2644718102762352),
 (1, 0.16707490095071303),
 (2, 0.24601861988302015),
 (3, 0.2074571051562886),
 (4, 0.23292587293757588),
 (5, 0.1281325374127274),
 (6, 0.22277034942870522),
 (7, 0.19601949215114584),
 (8, 0.1829267452057016),
 (9, 0.06532600014254476),
 (10, 0.19601949215114584),
 (11, 0.24601861988302015),
 (12, 0.2644718102762352),
 (13, 0.06828230983117356),
 (14, 0.2144726825443609),
 (15, 0.2644718102762352),
 (16, 0.15331774560157696),
 (17, 0.2144726825443609),
 (18, 0.13973298124401312),
 (19, 0.22277034942870522),
 (20, 0.2074571051562886),
 (21, 0.20137993559891662),
 (22, 0.17277122169683093),
 (23, 0.24601861988302015)]


In [138]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

In [139]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.042*"penni" + 0.041*"sheldon" + 0.035*"leonard" + 0.018*"wolowitz" + 0.018*"bernadett" + 0.012*"howard" + 0.010*"friend" + 0.009*"guy" + 0.009*"para" + 0.008*"help"
Topic: 1 
Words: 0.032*"sheldon" + 0.022*"howard" + 0.019*"bernadett" + 0.019*"leonard" + 0.011*"friend" + 0.011*"seri" + 0.011*"final" + 0.010*"penni" + 0.010*"get" + 0.010*"make"
Topic: 2 
Words: 0.035*"sheldon" + 0.023*"leonard" + 0.021*"date" + 0.012*"wolowitz" + 0.011*"howard" + 0.010*"apart" + 0.010*"take" + 0.008*"penni" + 0.008*"ask" + 0.007*"friend"
Topic: 3 
Words: 0.040*"leonard" + 0.035*"penni" + 0.034*"sheldon" + 0.018*"howard" + 0.013*"women" + 0.012*"bernadett" + 0.011*"talk" + 0.009*"siƒô" + 0.009*"comic" + 0.008*"decid"
Topic: 4 
Words: 0.016*"chip" + 0.014*"sheldon" + 0.011*"discuss" + 0.010*"joanna" + 0.009*"cameo" + 0.009*"famili" + 0.009*"texa" + 0.009*"coupl" + 0.008*"home" + 0.007*"hous"
Topic: 5 
Words: 0.042*"home" + 0.037*"texa" + 0.028*"famili" + 0.027*"waco" + 0.023*"chip" + 0.

In [140]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.007*"wolowitz" + 0.007*"sheldon" + 0.006*"wheaton" + 0.006*"look" + 0.006*"leonard" + 0.005*"reveal" + 0.005*"husband" + 0.005*"weekend" + 0.005*"spend" + 0.005*"secret"
Topic: 1 Word: 0.009*"sheldon" + 0.009*"leonard" + 0.007*"penni" + 0.007*"visit" + 0.006*"howard" + 0.006*"think" + 0.005*"bernadett" + 0.005*"live" + 0.005*"friend" + 0.005*"onlin"
Topic: 2 Word: 0.013*"advanc" + 0.011*"round" + 0.010*"game" + 0.009*"prize" + 0.009*"winner" + 0.009*"contest" + 0.008*"know" + 0.008*"chanc" + 0.008*"hand" + 0.008*"cash"
Topic: 3 Word: 0.005*"chip" + 0.005*"hecho" + 0.005*"sheldon" + 0.005*"gang" + 0.004*"question" + 0.004*"breakfast" + 0.004*"tournament" + 0.004*"race" + 0.004*"programa" + 0.004*"coverag"
Topic: 4 Word: 0.006*"penni" + 0.006*"naar" + 0.006*"continu" + 0.005*"high" + 0.005*"leav" + 0.005*"sheldon" + 0.005*"desd" + 0.005*"forc" + 0.005*"take" + 0.005*"home"
Topic: 5 Word: 0.007*"leonard" + 0.006*"bernadett" + 0.006*"state" + 0.006*"penni" + 0.005*"home" +

In [141]:
processed_docs[0]

0    [nerdi, physicist, share, apart, unlik, friend...
0    [hous, bedroom, space, turn, playroom, home, o...
0    [ellen, flair, hit, primetim, supers, version,...
Name: description, dtype: object

In [142]:
for index, score in sorted(lda_model[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.44466906785964966	 
Topic: 0.055*"sheldon" + 0.040*"leonard" + 0.037*"penni" + 0.014*"howard" + 0.011*"date" + 0.011*"game" + 0.010*"bernadett" + 0.010*"parti" + 0.009*"season" + 0.008*"get"

Score: 0.3416323959827423	 
Topic: 0.035*"sheldon" + 0.023*"leonard" + 0.021*"date" + 0.012*"wolowitz" + 0.011*"howard" + 0.010*"apart" + 0.010*"take" + 0.008*"penni" + 0.008*"ask" + 0.007*"friend"

Score: 0.18568678200244904	 
Topic: 0.018*"sheldon" + 0.014*"chip" + 0.012*"leonard" + 0.011*"home" + 0.010*"penni" + 0.009*"coupl" + 0.009*"pour" + 0.009*"compet" + 0.008*"contest" + 0.008*"joanna"


In [143]:
for index, score in sorted(lda_model_tfidf[bow_corpus[0]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.9639757871627808	 
Topic: 0.007*"leonard" + 0.006*"bernadett" + 0.006*"state" + 0.006*"penni" + 0.005*"home" + 0.005*"howard" + 0.005*"attempt" + 0.005*"sheldon" + 0.005*"date" + 0.005*"replay"


In [144]:
unseen_document = 'This house is up for sale or for escrow'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6999204754829407	 Topic: 0.023*"chip" + 0.016*"hous" + 0.014*"home" + 0.013*"sheldon" + 0.011*"waco"
Score: 0.0333574116230011	 Topic: 0.035*"sheldon" + 0.023*"leonard" + 0.021*"date" + 0.012*"wolowitz" + 0.011*"howard"
Score: 0.033345017582178116	 Topic: 0.018*"sheldon" + 0.014*"chip" + 0.012*"leonard" + 0.011*"home" + 0.010*"penni"
Score: 0.03334338217973709	 Topic: 0.042*"penni" + 0.041*"sheldon" + 0.035*"leonard" + 0.018*"wolowitz" + 0.018*"bernadett"
Score: 0.033343032002449036	 Topic: 0.042*"home" + 0.037*"texa" + 0.028*"famili" + 0.027*"waco" + 0.023*"chip"
Score: 0.033342961221933365	 Topic: 0.032*"sheldon" + 0.022*"howard" + 0.019*"bernadett" + 0.019*"leonard" + 0.011*"friend"
Score: 0.033338844776153564	 Topic: 0.016*"chip" + 0.014*"sheldon" + 0.011*"discuss" + 0.010*"joanna" + 0.009*"cameo"
Score: 0.03333630785346031	 Topic: 0.040*"leonard" + 0.035*"penni" + 0.034*"sheldon" + 0.018*"howard" + 0.013*"women"
Score: 0.03333630412817001	 Topic: 0.027*"advanc" + 0.026*"g

In [72]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
# This produces a feature matrix of token counts, similar to what
# CountVectorizer would produce on text.
X, _ = make_multilabel_classification(random_state=0)
lda = LatentDirichletAllocation(n_components=5,random_state=0)
lda.fit(X) 
# LatentDirichletAllocation(...)
# get topics for some given samples:
lda.transform(X[-2:])

array([[0.00360392, 0.25499205, 0.0036211 , 0.64236448, 0.09541846],
       [0.15297572, 0.00362644, 0.44412786, 0.39568399, 0.003586  ]])

In [76]:
X[0]

array([3., 1., 4., 0., 3., 0., 7., 2., 1., 0., 5., 3., 4., 6., 0., 2., 5.,
       4., 1., 3.])

In [77]:
_

array([[0, 0, 1, 1, 1],
       [0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 1, 1, 1, 1],
       [1, 1, 1, 0, 0],
       [1, 1, 1, 0, 0],
       [0, 1, 0, 0, 1],
       [0, 1, 1, 1, 1],
       [1, 1, 0, 0, 1],
       [1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 1, 0, 1],
       [0, 0, 0, 1, 1],
       [1, 1, 0, 1, 1],
       [0, 0, 1, 0, 0],
       [1, 0, 1, 1, 0],
       [1, 0, 0, 1, 1],
       [0, 0, 0, 1, 1],
       [0, 0, 1, 0, 1],
       [1, 1, 1, 1, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 1, 0],
       [1, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0],
       [1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [1, 1, 0, 1, 1],
       [0, 1, 0, 1, 0],
       [0, 1, 0, 0, 0],
       [0, 0, 1, 0, 0],
       [1, 1, 0, 1, 0],
       [1, 0, 0, 1, 0],
       [0, 1, 0, 1, 1],
       [0, 0, 1, 0, 1],
       [0, 0, 1, 0, 0],
       [0, 0, 0,